In [1]:
import torch
import torch.nn as nn
import numpy as np


3.1 logistic回归实战
在这一章里面，我们将处理一下结构化数据，并使用logistic回归对结构化数据进行简单的分类。

3.1.1 logistic回归介绍
logistic回归是一种广义线性回归（generalized linear model），与多重线性回归分析有很多相同之处。它们的模型形式基本上相同，都具有 wx + b，其中w和b是待求参数，其区别在于他们的因变量不同，多重线性回归直接将wx+b作为因变量，即y =wx+b,而logistic回归则通过函数L将wx+b对应一个隐状态p，p =L(wx+b),然后根据p 与1-p的大小决定因变量的值。如果L是logistic函数，就是logistic回归，如果L是多项式函数就是多项式回归。

说的更通俗一点，就是logistic回归会在线性回归后再加一层logistic函数的调用。

logistic回归主要是进行二分类预测，我们在激活函数时候讲到过 Sigmod函数，Sigmod函数是最常见的logistic函数，因为Sigmod函数的输出的是是对于0~1之间的概率值，当概率大于0.5预测为1，小于0.5预测为0。

下面我们就来使用公开的数据来进行介绍

3.1.2 UCI German Credit 数据集
UCI German Credit是UCI的德国信用数据集，里面有原数据和数值化后的数据。

German Credit数据是根据个人的银行贷款信息和申请客户贷款逾期发生情况来预测贷款违约倾向的数据集，数据集包含24个维度的，1000条数据，

在这里我们直接使用处理好的数值化的数据，作为展示。

地址

3.2 代码实战
我们这里使用的 german.data-numeric是numpy处理好数值化数据，我们直接使用numpy的load方法读取即可

In [2]:
data = np.loadtxt('german.data-numeric')

In [3]:
#标准化
n,l = data.shape
for j in range(l-1):
    meanVal = np.mean(data[:,j])
    stdVal = np.std(data[:,j])
    data[:,j] = (data[:,j]-meanVal)/stdVal

In [5]:
data


array([[-1.25456565, -1.23647786,  1.34401408, ..., -0.5       ,
         0.76635604,  1.        ],
       [-0.45902624,  2.24819436, -0.50342796, ..., -0.5       ,
         0.76635604,  2.        ],
       [ 1.13205258, -0.73866754,  1.34401408, ...,  2.        ,
        -1.30487651,  1.        ],
       ...,
       [ 1.13205258, -0.73866754, -0.50342796, ..., -0.5       ,
         0.76635604,  1.        ],
       [-1.25456565,  1.9992892 , -0.50342796, ..., -0.5       ,
         0.76635604,  2.        ],
       [-0.45902624,  1.9992892 ,  1.34401408, ..., -0.5       ,
         0.76635604,  1.        ]])

In [6]:
#打乱数据
n

1000

In [7]:
l

25

In [8]:
data[:,1]

array([-1.23647786,  2.24819436, -0.73866754,  1.75038404,  0.25695309,
        1.25257373,  0.25695309,  1.25257373, -0.73866754,  0.75476341,
       -0.73866754,  2.24819436, -0.73866754,  0.25695309, -0.48976238,
        0.25695309,  0.25695309,  0.75476341,  0.25695309,  0.25695309,
       -0.9875727 , -1.23647786, -0.90460432, -0.73866754, -0.90460432,
       -1.23647786, -1.23647786, -0.73866754, -1.15350947,  3.243815  ,
       -0.24085723,  0.25695309, -0.24085723, -0.73866754, -0.73866754,
        1.9992892 ,  2.24819436, -0.24085723, -0.90460432, -0.9875727 ,
        0.75476341, -0.73866754, -0.24085723,  0.75476341,  2.24819436,
       -0.82163593,  1.25257373, -1.23647786, -0.82163593, -0.73866754,
        0.25695309,  0.50585825, -0.73866754, -0.24085723,  1.25257373,
       -1.23647786, -0.73866754,  1.25257373, -0.24085723,  1.25257373,
       -0.9875727 , -0.48976238,  1.25257373,  2.24819436,  0.25695309,
        0.50585825, -0.73866754, -0.73866754,  1.25257373,  1.25

In [12]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std_data = std.fit_transform(data)

In [13]:
std_data

array([[-1.25456565, -1.23647786,  1.34401408, ..., -0.5       ,
         0.76635604, -0.65465367],
       [-0.45902624,  2.24819436, -0.50342796, ..., -0.5       ,
         0.76635604,  1.52752523],
       [ 1.13205258, -0.73866754,  1.34401408, ...,  2.        ,
        -1.30487651, -0.65465367],
       ...,
       [ 1.13205258, -0.73866754, -0.50342796, ..., -0.5       ,
         0.76635604, -0.65465367],
       [-1.25456565,  1.9992892 , -0.50342796, ..., -0.5       ,
         0.76635604,  1.52752523],
       [-0.45902624,  1.9992892 ,  1.34401408, ..., -0.5       ,
         0.76635604, -0.65465367]])

In [14]:
#打乱数据
np.random.shuffle(data)

区分训练集和测试集，由于这里没有验证集，所以我们直接使用测试集的准确度作为评判好坏的标准

区分规则：900条用于训练，100条作为测试

german.data-numeric的格式为，前24列为24个维度，最后一个为要打的标签（0，1），所以我们将数据和标签一起区分出来

In [20]:
train_data = data[:900,:l-1]
train_lab = data[:900,l-1]-1
test_data = data[900:,:l-1]
test_lab = data[900:,l-1]-1

In [21]:
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.fc = nn.Linear(24,2)
    def forward(self,x):
        out = self.fc(x)
        out = torch.sigmoid(out)
        return out


In [22]:
def test(pred,lab):
    t = pred.max(-1)[1] == lab
    return torch.mean(t.float())

In [23]:
net = LR()
criterion =nn.CrossEntropyLoss()
optm = torch.optim.Adam(net.parameters())
epochs = 1000

for i in range(epochs):
    net.train()#指定模式为训练模式
    x = torch.from_numpy(train_data).float()
    y = torch.from_numpy(train_lab).long()
    y_hat = net(x)
    loss = criterion(y_hat,y)
    optm.zero_grad()
    loss.backward()
    optm.step()
    if (i+1)%100 == 0:
        net.eval()#指定模式为计算模式
        test_in = torch.from_numpy(test_data).float()
        test_l = torch.from_numpy(test_lab).long()
        test_out = net(test_in)
        accu=test(test_out,test_l)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.2f}".format(i+1,loss.item(),accu))

Epoch:100,Loss:0.6421,Accuracy：0.68
Epoch:200,Loss:0.6139,Accuracy：0.74
Epoch:300,Loss:0.5972,Accuracy：0.75
Epoch:400,Loss:0.5858,Accuracy：0.80
Epoch:500,Loss:0.5772,Accuracy：0.82
Epoch:600,Loss:0.5705,Accuracy：0.82
Epoch:700,Loss:0.5650,Accuracy：0.82
Epoch:800,Loss:0.5604,Accuracy：0.82
Epoch:900,Loss:0.5564,Accuracy：0.82
Epoch:1000,Loss:0.5530,Accuracy：0.82


In [24]:
test_data

array([[ 1.13205258, -0.82163593, -0.50342796, ..., -0.14998296,
        -0.5       ,  0.76635604],
       [-0.45902624,  1.25257373, -0.50342796, ..., -0.14998296,
        -0.5       , -1.30487651],
       [-0.45902624, -0.24085723, -0.50342796, ..., -0.14998296,
        -0.5       ,  0.76635604],
       ...,
       [ 0.33651317,  0.75476341,  1.34401408, ..., -0.14998296,
         2.        , -1.30487651],
       [ 1.13205258,  0.25695309, -0.50342796, ..., -0.14998296,
        -0.5       ,  0.76635604],
       [-1.25456565, -0.24085723, -1.42714898, ..., -0.14998296,
        -0.5       , -1.30487651]])